<a href="https://colab.research.google.com/github/VesleAnne/Diploma/blob/main/Notebooks/Text_preprocessing_and_enbeddings3_SbertDistil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Perform Google Colab installs
import os

if "COLAB_GPU" in os.environ:
    print("[INFO] Running in Google Colab, installing requirements.")
    !pip install -U torch -q # requires torch 2.1.1+ (for efficient sdpa implementation)
#    !pip install PyMuPDF # for reading PDFs with Python if needed
    !pip install python-docx -q #for reading Word with Python
    !pip install tqdm -q # for progress bars
    !pip install sentence-transformers -q # for embedding models
    !pip install accelerate -q # for quantization model loading
    !pip install bitsandbytes -q # for quantizing models (less storage space)
    !pip install flash-attn --no-build-isolation -q # for faster attention mechanism = faster LLM inference

[INFO] Running in Google Colab, installing requirements.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 7.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.3.0 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.3.0 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.3.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 6.8 MB/s eta 0:00:0

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
from docx import Document
import random
import pandas as pd
import torch
from tqdm.auto import tqdm
import re
import numpy as np
from sentence_transformers import util, SentenceTransformer

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
%cd '/content/drive/MyDrive/МФТИ/вкр/RAG'

/content/drive/MyDrive/МФТИ/вкр/RAG


In [ ]:
def text_formatter(text: str) -> str:
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip()  # Replace newline characters with spaces and strip leading/trailing whitespace
    return cleaned_text

In [ ]:
def open_and_read_word(docx_path: str) -> list[dict]:
    """
    Opens a Word document, reads its content page by page, and collects statistics.

    Parameters:
        docx_path (str): The file path to the Word document to be opened and read.

    Returns:
        list[dict]: A list of dictionaries, each containing the page number
        (adjusted), character count, word count, sentence count, token count, and the extracted text
        for each page.
    """
    doc = Document(docx_path)  # Open the Word document using python-docx
    pages_and_texts = []
    for page_number, paragraph in enumerate(doc.paragraphs):
        text = paragraph.text.strip()  # Get text from the paragraph and remove leading/trailing whitespace
        if text:  # Exclude empty paragraphs
            text = text_formatter(text)  # Format the text using the text_formatter function
            pages_and_texts.append({"page_number": page_number,
                                    "page_char_count": len(text),
                                    "page_word_count": len(text.split()),  # Count words by splitting on whitespace
                                    "page_sentence_count_raw": len(text.split(". ")),  # Count sentences by splitting on periods followed by spaces
                                    "page_token_count": len(text) / 4,  # Estimate token count
                                    "text": text})
    return pages_and_texts

In [ ]:
# Open and read the Word document, extracting text and collecting statistics
docx_path = '/content/drive/MyDrive/МФТИ/вкр/RAG/DATA/greendata.docx'
pages_and_texts = open_and_read_word(docx_path=docx_path)

In [ ]:
random.sample(pages_and_texts, k=3)

[{'page_number': 3334,
  'page_char_count': 12,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 3.0,
  'text': 'PHONE_NUMBER'},
 {'page_number': 1602,
  'page_char_count': 5,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 1.25,
  'text': 'image'},
 {'page_number': 3366,
  'page_char_count': 12,
  'page_word_count': 2,
  'page_sentence_count_raw': 1,
  'page_token_count': 3.0,
  'text': 'Номера отеля'}]

In [ ]:
# Read Excel files
excel1_path = '/content/drive/MyDrive/МФТИ/вкр/RAG/DATA/faq.xlsx'
#excel2_path = '/content/drive/MyDrive/МФТИ/вкр/RAG/DATA/данные2.xlsx'

In [ ]:
df_excel1 = pd.read_excel(excel1_path)
df_excel1.head()

,Тема,Вопрос,Схема ответа,Кнопка ссылки на вики,Ссылка на wiki,Тэги для поиска
0,Личный кабинет,Как зарегистрироваться в ЛК?,1. Для регистрации в личном кабинете перейдите...,Регистрация в ЛК,https://wiki.greendata.store/%D0%A0%D0%B5%D0%B...,"Личный кабинет, регистрация, регистрация ЛК SA..."
1,Личный кабинет,Зачем указывать страну? |Зачем выбрирать язык?...,Кнопка с флагом в правом верхнем углу нужна дл...,NaN,NaN,"Личный кабинет, язык, не тот язык, настройка я..."
2,Личный кабинет,Не приходит подтверждение на почту,I. Если Вам не приходит письмо для подтвержден...,NaN,NaN,"Личный кабинет, регистрация, подтверждение рег..."
3,Личный кабинет,Нет почты/забыл почту. Есть ли другой канал дл...,В настоящее время доступна регистрация только ...,NaN,NaN,"Личный кабинет, регистрация, регистрация ЛК SA..."
4,Личный кабинет,Почему у меня все отображается не на том языке?,Выбор языка зависит от настроек браузера. \nCh...,NaN,NaN,"Личный кабинет, язык, не тот язык, настройка я..."


In [ ]:
#df_excel2 = pd.read_excel(excel2_path).dropna()
#df_excel2.head()

In [ ]:
text_excel1 = [text_formatter(str(text)) for text in df_excel1['Схема ответа']]
#text_excel2 = [text_formatter(str(text)) for text in df_excel2['text']]

In [ ]:
#all_text = text_excel1 + text_excel2

In [ ]:
#def remove_smileys_and_links(text):
#    text = re.sub(r':-?[)DPO()]|;-?[/|]', '', text)
#    text = re.sub(r'jivo media_guid_[\w-]+', '', text)
#    text = re.sub(r'🖼', '', text)
#    text = re.sub(r'👍', '', text)
#    text = re.sub(r'🐳', '', text)
#    text = re.sub(r'👋', '', text)
#    return text

#cleaned_all_text = [remove_smileys_and_links(text) for text in all_text]
#cleaned_all_text = list(filter(lambda x: x.strip(), cleaned_all_text))

In [ ]:
# Добавляем фразу "Переключаю на оператора" в конец списка 20 раз
text_excel1.extend(["Переключаю на оператора"] * 20)

In [ ]:
# Создаём новый датафрейм с текстом из Excel файлов
df_new = pd.DataFrame({"text": text_excel1})
df_new.head()

,text
0,1. Для регистрации в личном кабинете перейдите...
1,Кнопка с флагом в правом верхнем углу нужна дл...
2,I. Если Вам не приходит письмо для подтвержден...
3,В настоящее время доступна регистрация только ...
4,Выбор языка зависит от настроек браузера. Chr...


In [ ]:
# Для каждой строки в датафрейме вычисляем статистики
for idx, row in df_new.iterrows():
    text = row['text']
    # Подсчитываем статистики для каждой строки

    char_count = len(text)
    word_count = len(text.split())
    sentence_count = len(text.split("."))
    token_count = len(text) / 4  # Примерное количество токенов

    # Записываем статистики обратно в датафрейм
    df_new.loc[idx, 'page_number'] = 145
    df_new.loc[idx, 'page_char_count'] = char_count
    df_new.loc[idx, 'page_word_count'] = word_count
    df_new.loc[idx, 'page_sentence_count_raw'] = sentence_count
    df_new.loc[idx, 'page_token_count'] = token_count


In [ ]:
df_new.head()

,text,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
0,1. Для регистрации в личном кабинете перейдите...,145.0,335.0,45.0,8.0,83.75
1,Кнопка с флагом в правом верхнем углу нужна дл...,145.0,138.0,22.0,3.0,34.50
2,I. Если Вам не приходит письмо для подтвержден...,145.0,431.0,60.0,8.0,107.75
3,В настоящее время доступна регистрация только ...,145.0,67.0,9.0,2.0,16.75
4,Выбор языка зависит от настроек браузера. Chr...,145.0,718.0,108.0,9.0,179.50


In [ ]:
cols = df_new.columns.tolist()
cols = cols[1:] + [cols[0]]
df_new = df_new[cols]

df_new.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,145.0,335.0,45.0,8.0,83.75,1. Для регистрации в личном кабинете перейдите...
1,145.0,138.0,22.0,3.0,34.50,Кнопка с флагом в правом верхнем углу нужна дл...
2,145.0,431.0,60.0,8.0,107.75,I. Если Вам не приходит письмо для подтвержден...
3,145.0,67.0,9.0,2.0,16.75,В настоящее время доступна регистрация только ...
4,145.0,718.0,108.0,9.0,179.50,Выбор языка зависит от настроек браузера. Chr...


In [ ]:
df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,0,35,7,1,8.75,С чего начать при работе с Системой
1,1,198,23,2,49.50,"Вы начинаете работу с Системой GreenData, кото..."
2,3,45,6,1,11.25,"быстрее внедрять нужные сервисы и приложения,"
3,4,57,8,1,14.25,не согласовывая стек технологий и правила обме...
4,6,49,6,1,12.25,"вносить любые изменения в информационную систему,"


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2014 entries, 0 to 2013
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   page_number              2014 non-null   int64  
 1   page_char_count          2014 non-null   int64  
 2   page_word_count          2014 non-null   int64  
 3   page_sentence_count_raw  2014 non-null   int64  
 4   page_token_count         2014 non-null   float64
 5   text                     2014 non-null   object 
dtypes: float64(1), int64(4), object(1)
memory usage: 94.5+ KB


In [ ]:
df = pd.concat([df, df_new], ignore_index=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2307 entries, 0 to 2306
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   page_number              2307 non-null   float64
 1   page_char_count          2307 non-null   float64
 2   page_word_count          2307 non-null   float64
 3   page_sentence_count_raw  2307 non-null   float64
 4   page_token_count         2307 non-null   float64
 5   text                     2307 non-null   object 
dtypes: float64(5), object(1)
memory usage: 108.3+ KB


In [ ]:
# Преобразование датафрейма в словарь
pages_and_texts = df.to_dict(orient='records')

In [ ]:
pages_and_texts

[{'page_number': 0.0,
  'page_char_count': 35.0,
  'page_word_count': 7.0,
  'page_sentence_count_raw': 1.0,
  'page_token_count': 8.75,
  'text': 'С чего начать при работе с Системой'},
 {'page_number': 1.0,
  'page_char_count': 198.0,
  'page_word_count': 23.0,
  'page_sentence_count_raw': 2.0,
  'page_token_count': 49.5,
  'text': 'Вы начинаете работу с Системой GreenData, которая содержит передовой функционал автоматизации и способна преобразить работу любого бизнеса. Благодаря передовым инструментам автоматизации Вы сможете:'},
 {'page_number': 3.0,
  'page_char_count': 45.0,
  'page_word_count': 6.0,
  'page_sentence_count_raw': 1.0,
  'page_token_count': 11.25,
  'text': 'быстрее внедрять нужные сервисы и приложения,'},
 {'page_number': 4.0,
  'page_char_count': 57.0,
  'page_word_count': 8.0,
  'page_sentence_count_raw': 1.0,
  'page_token_count': 14.25,
  'text': 'не согласовывая стек технологий и правила обмена данными.'},
 {'page_number': 6.0,
  'page_char_count': 49.0,
  'p

In [ ]:
# Download NLTK's punkt package, used for tokenization
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Loop through each page and text extracted from the Word document
for item in tqdm(pages_and_texts):
    # Tokenize text into sentences using NLTK
    item["sentences"] = sent_tokenize(item["text"])

    # Ensure all sentences are represented as strings
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]

    # Count the number of sentences
    item["page_sentence_count_nltk"] = len(item["sentences"])

  0%|          | 0/2307 [00:00<?, ?it/s]

In [ ]:
# Create a DataFrame to get stats
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_nltk
count,2307.00,2307.00,2307.00,2307.00,2307.00,2307.00
mean,1635.12,95.09,12.16,1.52,23.77,1.41
std,1156.29,114.69,14.83,1.42,28.67,1.19
min,0.00,1.00,1.00,1.00,0.25,1.00
25%,514.00,19.00,2.00,1.00,4.75,1.00
50%,1586.00,58.00,7.00,1.00,14.50,1.00
75%,2637.00,131.00,17.00,1.00,32.75,1.00
max,3729.00,1615.00,208.00,19.00,403.75,16.00


In [ ]:
# Define split size to turn groups of sentences into chunks
num_sentence_chunk_size = 10

# Create a function that recursively splits a list into desired sizes
def split_list(input_list: list,
               slice_size: int) -> list[list[str]]:
    """
    Splits the input_list into sublists of size slice_size (or as close as possible).

    For example, a list of 17 sentences would be split into two lists of [[10], [7]]
    """
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

# Loop through pages and texts and split sentences into chunks
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/2307 [00:00<?, ?it/s]

In [ ]:
random.sample(pages_and_texts, k=1)

[{'page_number': 1534.0,
  'page_char_count': 156.0,
  'page_word_count': 20.0,
  'page_sentence_count_raw': 2.0,
  'page_token_count': 39.0,
  'text': 'Когда все данные введены корректно, следует нажать на кнопку "Подтвердить реквизиты". После чего над кнопкой отобразится надпись "Данные успешно сохранены".',
  'sentences': ['Когда все данные введены корректно, следует нажать на кнопку "Подтвердить реквизиты".',
   'После чего над кнопкой отобразится надпись "Данные успешно сохранены".'],
  'page_sentence_count_nltk': 2,
  'sentence_chunks': [['Когда все данные введены корректно, следует нажать на кнопку "Подтвердить реквизиты".',
    'После чего над кнопкой отобразится надпись "Данные успешно сохранены".']],
  'num_chunks': 1}]

In [ ]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_nltk,num_chunks
count,2307.00,2307.00,2307.00,2307.00,2307.00,2307.00,2307.00
mean,1635.12,95.09,12.16,1.52,23.77,1.41,1.00
std,1156.29,114.69,14.83,1.42,28.67,1.19,0.05
min,0.00,1.00,1.00,1.00,0.25,1.00,1.00
25%,514.00,19.00,2.00,1.00,4.75,1.00,1.00
50%,1586.00,58.00,7.00,1.00,14.50,1.00,1.00
75%,2637.00,131.00,17.00,1.00,32.75,1.00,1.00
max,3729.00,1615.00,208.00,19.00,403.75,16.00,2.00


In [ ]:

# Split each chunk into its own item
pages_and_chunks = []
for item in tqdm(pages_and_texts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]

        # Join the sentences together into a paragraph-like structure, aka a chunk (so they are a single string)
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) # ".A" -> ". A" for any full-stop/capital letter combo
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        # Get stats about the chunk
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 # 1 token = ~4 characters

        pages_and_chunks.append(chunk_dict)

# How many chunks do we have?
len(pages_and_chunks)

  0%|          | 0/2307 [00:00<?, ?it/s]

2312

In [ ]:
# View a random sample
random.sample(pages_and_chunks, k=1)

[{'page_number': 2673.0,
  'sentence_chunk': 'Рабочий/ выходной день',
  'chunk_char_count': 22,
  'chunk_word_count': 3,
  'chunk_token_count': 5.5}]

In [ ]:
# Get stats about our chunks
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,2312.00,2312.00,2312.00,2312.00
mean,1631.90,94.35,11.77,23.59
std,1157.11,111.23,14.12,27.81
min,0.00,1.00,1.00,0.25
25%,507.50,19.00,2.00,4.75
50%,1581.00,58.00,7.00,14.50
75%,2634.50,131.25,16.00,32.81
max,3729.00,1319.00,181.00,329.75


In [ ]:
# Show random chunks with under 30 tokens in length
min_token_length = 30
for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk token count: 7.5 | Text: выполнять настройку схемы ETL;
Chunk token count: 1.25 | Text: image
Chunk token count: 5.0 | Text: Проекты и приложения
Chunk token count: 3.0 | Text: Версионность
Chunk token count: 1.25 | Text: Шрифт


In [ ]:
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

[{'page_number': 1.0,
  'sentence_chunk': 'Вы начинаете работу с Системой GreenData, которая содержит передовой функционал автоматизации и способна преобразить работу любого бизнеса.Благодаря передовым инструментам автоматизации Вы сможете:',
  'chunk_char_count': 197,
  'chunk_word_count': 22,
  'chunk_token_count': 49.25},
 {'page_number': 13.0,
  'sentence_chunk': 'Постепенно Вы научитесь эффективно применять весь инструментарий платформы GreenData, а сейчас ознакомьтесь с предназначением основных модулей.',
  'chunk_char_count': 143,
  'chunk_word_count': 16,
  'chunk_token_count': 35.75}]

In [ ]:
embedding_model = SentenceTransformer(model_name_or_path="FractalGPT/SbertDistil",
                                      device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.58k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/706k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/482k [00:00<?, ?B/s]

In [ ]:
%%time

# Send the model to the GPU
embedding_model.to(device) # requires a GPU installed, for reference on my local machine, I'm using a NVIDIA RTX 4090

# Create embeddings one by one on the GPU
for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])

  0%|          | 0/653 [00:00<?, ?it/s]

CPU times: user 4.78 s, sys: 136 ms, total: 4.92 s
Wall time: 5.27 s


In [ ]:
# Turn text chunks into a single list
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]

In [ ]:
%%time

# Embed all texts in batches
text_chunk_embeddings = embedding_model.encode(text_chunks,
                                               batch_size=32, # you can use different batch sizes here for speed/performance, I found 32 works well for this use case
                                               convert_to_tensor=True) # optional to return embeddings as tensor instead of array

text_chunk_embeddings

CPU times: user 397 ms, sys: 12.2 ms, total: 409 ms
Wall time: 342 ms


tensor([[ 0.0322, -0.1441, -0.0157,  ...,  0.2498,  0.3240, -0.0065],
        [ 0.2265, -0.1641, -0.0342,  ...,  0.2128,  0.1285,  0.0989],
        [-0.1788,  0.1183, -0.4150,  ...,  0.0453,  0.0726, -0.1894],
        ...,
        [ 0.0126, -0.0116, -0.1191,  ...,  0.1234,  0.0774,  0.1429],
        [-0.1225, -0.0056, -0.1278,  ...,  0.2131, -0.0720,  0.0056],
        [ 0.0636, -0.3934, -0.1514,  ...,  0.2468,  0.2204,  0.1064]],
       device='cuda:0')

In [ ]:
# Save embeddings to file
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [ ]:
%cd '/content/drive/MyDrive/МФТИ/вкр/RAG'

/content/drive/MyDrive/МФТИ/вкр/RAG


In [ ]:
current_directory = os.getcwd()
# Save embeddings to file
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = os.path.join(current_directory, 'DATA', 'text_chunks_and_embeddings_df3.csv')
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [ ]:
# Import saved file and view
text_chunks_and_embedding_df_load = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embedding_df_load.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,1.0,"Вы начинаете работу с Системой GreenData, кото...",197,22,49.25,[ 3.22012901e-02 -1.44123971e-01 -1.56841502e-...
1,13.0,Постепенно Вы научитесь эффективно применять в...,143,16,35.75,[ 2.26459116e-01 -1.64134279e-01 -3.42093185e-...
2,18.0,"Модуль ""Модель данных"" позволяет организовать ...",168,20,42.00,[-0.17881122 0.11829569 -0.4149923 -0.279280...
3,38.0,"Возможности модуля ""Бизнес-процессы"" позволяют...",148,16,37.00,[-1.69715032e-01 8.36630538e-02 -1.32400841e-...
4,60.0,В платформе GreenData логика работы строится н...,203,24,50.75,[ 0.06771126 -0.10491905 -0.31014097 -0.237664...
